In [53]:
import csv
import pandas as pd
import hvplot.pandas
import matplotlib as plt
from pprint import pprint
import kagglehub
import os
import requests


# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/most-streamed-spotify-songs-2024")
if not os.path.exists(os.path.join("data", "spotify.csv")):
    file_name = os.listdir(path)[0]
    os.rename(os.path.join(path, file_name), os.path.join("data", "spotify.csv"))
spotify_df = pd.read_csv("data/spotify.csv", encoding='ISO-8859-1')

In [55]:
clean_df = spotify_df.drop(columns=['TIDAL Popularity'])
# clean_df = clean_df.dropna()
for i in clean_df.columns:
    clean_df.rename(columns={i: (i.lower()).replace(" ", "_")}, inplace=True)
clean_df.describe()

,track_score,spotify_popularity,apple_music_playlist_count,deezer_playlist_count,amazon_playlist_count,explicit_track
count,4600.000000,3796.000000,4039.00000,3679.000000,3545.000000,4600.000000
mean,41.844043,63.501581,54.60312,32.310954,25.348942,0.358913
std,38.543766,16.186438,71.61227,54.274538,25.989826,0.479734
min,19.400000,1.000000,1.00000,1.000000,1.000000,0.000000
25%,23.300000,61.000000,10.00000,5.000000,8.000000,0.000000
50%,29.900000,67.000000,28.00000,15.000000,17.000000,0.000000
75%,44.425000,73.000000,70.00000,37.000000,34.000000,1.000000
max,725.400000,96.000000,859.00000,632.000000,210.000000,1.000000


In [57]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
lastfm_id = os.getenv("lastfm_id")
lastfm_secret = os.getenv("lastfm_secret")

lastfm_url = 'http://ws.audioscrobbler.com/2.0/'
parameters = {
    'method': 'track.getTopTags',
    'api_key': lastfm_id,
    'format': 'json',
    'autocorrect': 1
}
if 'lastfm_tags' not in clean_df.columns:
    clean_df['lastfm_tags'] = None
else: 
    for index, row in clean_df[clean_df['lastfm_tags'].apply(lambda x: x == [] or x == None)].iterrows():
        parameters['track'] = row['track']
        parameters['artist'] = row['artist']
        r = requests.get(lastfm_url, params=parameters).json()
        try:
            tags = [tag['name'] for tag in r['toptags']['tag']]
        except KeyError:
            print(f"Error with {row['track']} by {row['artist']}")
            pprint(r)
            tags = []
        clean_df.at[index, 'lastfm_tags'] = tags# Initialize the column with None if it doesn't exist or has empty values


Error with TEXAS HOLD 'EM by Beyoncï¿
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe by sped up 8282
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Dona Felicidade by Patati Patatï¿
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with CARNIVAL by ýý$
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Laxed ï¿½ï¿½ï¿½ Sire by Jawsh 685
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Titï¿½ï¿½ Me Pregu by Bad Bunny
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Prada by cassï¿
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Hands by Dianï¿½
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with Chegou a Hora de Ir para Cama (Playback) by 3 Palavrinhas
{'error': 6, 'links': [], 'message': 'Track not found'}
Error with MI EX TENï¿½ï¿½A RA by KAROL G
{'error': 6, 'links': [], 'message': 'Track not fou

In [ ]:
# parameters = {
#     'method': 'track.search',
#     'api_key': lastfm_id,
#     'format': 'json',
#     'autocorrect': 1
# }
# empty_tags_rows = clean_df[clean_df['lastfm_tags'].apply(lambda x: x == [])]
# requests.get(lastfm_url+empty_tags_rows['track'][127], params=parameters).url

'http://ws.audioscrobbler.com/2.0/MI%20EX%20TEN%C3%AF%C2%BF%C2%BD%C3%AF%C2%BF%C2%BDA%20RA?method=track.search&api_key=e708bc383713f92c7b99d4d735b44ef4&format=json&autocorrect=1'

In [43]:
df = clean_df[clean_df['lastfm_tags'].apply(lambda x: x != [])]
df.reset_index(inplace=True)
df.rename(columns={'index': 'original_rank'}, inplace=True)

/var/folders/ph/vmwntg591nz4p53wxs1w08dr0000gn/T/ipykernel_80850/2659353285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'index': 'original_rank'}, inplace=True)


In [52]:
def tag_frequency(tag, df):
    """
    Calculate the frequency of a specific tag in the DataFrame.

    Parameters:
    tag (str): The tag to search for in the 'lastfm_tags' column.
    df (pd.DataFrame): The DataFrame containing the 'lastfm_tags' column.

    Returns:
    int: The number of occurrences of the specified tag in the DataFrame.
    
    Example usage:
    tag = 'Hip-Hop'
    frequency = tag_frequency(tag, clean_df)
    print(f"The tag '{tag}' appears {frequency} times in the lastfm_tags column.")
    """
    return df['lastfm_tags'].apply(lambda x: tag in x).sum() # df['lastfm_tags'].apply(lambda x: tag in x) returns a boolean series with true for each row that contains the tag 

all_tags = clean_df['lastfm_tags'].explode().unique()

tag_frequency_df = pd.DataFrame({'tag': all_tags, 'frequency': [tag_frequency(tag, df) for tag in all_tags]})

tag_frequency_df.sort_values('frequency', ascending=False, inplace=True)
tag_frequency_df

,tag,frequency
7,pop,228
34,MySpotigramBot,174
400,2010s,144
1,rap,132
0,rnb,129
...,...,...
2315,describes me,1
2317,tired,1
2320,For Her,1
4618,trending tracks,1
